In [ ]:
import sys, os
cwd = os.getcwd()
frameworkDir = os.path.abspath(os.path.join(cwd, os.pardir, os.pardir))
sys.path.append(frameworkDir)

import pandas as pd

from dackar.knowledge_graph.KGconstruction import KG

In [ ]:
cause_dict= {'A': ['alpha','beta','gamma'],   
             'B': [1.,2.,3.]}
cause_df = pd.DataFrame(cause_dict)

In [ ]:
config_file_path = '/Users/mandd/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-666d9aa2-b0c5-4903-9f08-1cad30136945/conf/neo4j.conf'
uri = "bolt://localhost:7687"
pwd = "123456789"
user='neo4j'

knowledgeGraph = KG(config_file_path, uri, pwd, user)

knowledgeGraph.resetGraph()

In [ ]:
knowledgeGraph.importGraphSchema('condition_report','schemas/conditionReportSchema.toml')

In [ ]:
nodeConstSchemaCause  = {'cause': {'prop1': 'A', 'prop2': 'B'}}
constSchemaCause = {'nodes': nodeConstSchemaCause}

knowledgeGraph.genericWorkflow(cause_df, constSchemaCause)

In [ ]:
conseq_dict = {'C': ['x','y','z'],           
               'D': [1.1,1.2,1.3],
               'prop_rel': [2.1,2.2,2.3],
               'A': ['alpha','beta','gamma']}   # link to cause
conseq_df = pd.DataFrame(conseq_dict)

nodeConstSchemaConseq = {'consequence': {'ID_A': 'C', 'ID_B': 'D'}}
relConstSchemaConseq = [{'source': {'cause.prop1':'A'},
                         'target': {'consequence.ID_A':'C'},
                         'type'  : 'causality',
                         'properties': {'amplitude': 'prop_rel'}}] 

constSchemaConseq= {'nodes'    : nodeConstSchemaConseq,
                    'relations': relConstSchemaConseq}

knowledgeGraph.genericWorkflow(conseq_df, constSchemaConseq)

In [ ]:
node_properties=[{'name': 'date', 'type': 'string', 'optional': False},
                 {'name': 'ID', 'type': 'string', 'optional': False}]

df = pd.DataFrame(node_properties)
df